# Activity 9

This activity tests your skills
in approaching an engineering problem
as solving a system of nonlinear algebraic equations.

## Scenario

Referring to [the diagram](./act-09.png),
a device has an active power consumption $P$
when operating at a lagging power factor $\Phi$.
The requisite power is conveyed by the
resistance $R$,
inductance $L$
and the capacitor $C$,
from a source
configured to provide a steady RMS voltage $U$
at a steady frequency $F$.

You are to solve for the following quantities.
- $V$, the RMS operating voltage of the device
- $\delta_{\text{l}}$,
  the phase angle of the voltage phasor at which the device receives power
- $\delta_{\text{s}}$
  the phase angle of the voltage phasor provided by the source

## Modelling

Cast the original task as
solving a system of nonlinear algebraic equations
in three unknowns
(namely, $V$, $\delta_{\text{l}}$, and $\delta_{\text{s}}$).
Introduce a vector $\boldsymbol{x}$ 
to collect the unknowns in the order presented above,
so that the system of equations can be written in vector form as
$\boldsymbol{f}\!\left(\boldsymbol{x}\right) = \boldsymbol{0}$.
Set the voltage phasor across the input terminals of the load
to have the reference phase angle.

## Core code

In [1]:
import math as mt

import numpy as np
import scipy as sp
from numpy import pi, sin, cos, tan, acos
from numpy import pi
from scipy.optimize import root

Define a Python function
`f()` that takes

- one positional argument,
  `x`, representing $\boldsymbol{x}$;
  and
- four keyword arguments,
  `U`, `F`, `R`, `L`, `C`, `P`, and `Phi`
  representing $U$, $F$, $R$, $L$, $C$, $P$, and $\Phi$ respectively.

`f()` returns a NumPy array representing
$\boldsymbol{f}\!\left(\boldsymbol{x}\right)$.

Moreover, define a Python function `dfdx()`
that takes the same arguments as `f()`,
and returns a NumPy array representing
the Jacobian of $\boldsymbol{f}\!\left(\boldsymbol{x}\right)$ w.r.t. $\boldsymbol{x}$.


In [2]:
def f(x, U, F, R, L, C, P, Phi):
    
    Pw = P * 1e-3 # Convert power from mW to W

    V = x[0]
    theta = x[1]

    Q = Pw * tan(acos(Phi))
    w = 2 * pi * F

    f1 = (
        R * (Pw / V)
        - (w * L / V) * (Q - V**2 * w * C)
        - U * cos(theta)
        + V
    )

    f2 = (
        (R / V) * (Q - V**2 * w * C)
        + (w * L * Pw) / V
        - U * sin(theta)
    )

    return np.array([f1, f2])


def dfdx(x, U, F, R, L, C, P, Phi):
    Pw = P * 1e-3

    V = x[0]
    theta = x[1]

    Q = Pw * tan(acos(Phi))
    w = 2 * pi * F

    df1dV = (
        (Pw * R) / V**2
        - (w * L / V**2) * (Q - V**2 * w * C)
        + 2 * (w * L) * w * C * V
        + 1
    )

    df1dtheta = U * sin(theta)

    df2dV = (
        -R * (Q - V**2 * w * C) / V**2
        - 2 * R * w * C
        - (w * L * Pw) / V**2
    )

    df2dtheta = -U * cos(theta)

    return np.array([
        [df1dV, df1dtheta],
        [df2dV, df2dtheta]
    ])

## Assessment

Every time you solve for $\boldsymbol{x}$,
- define Python variables
  `U`, `F`, `R`, `L`, `C`, `P`, and `PHI`
  for storing the values of parameters
  $U$, $F$, $R$, $L$, $C$, $P$, and $\phi$,
  respectively;
- define a Python variable `x_ini`
  for storing your initial guesses of the variables;
- use [`scipy.optimize.root()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.root.html),
  particularly its `args` keyword argument;
- define a Python variable `x_out` to store $\boldsymbol{x}$;
- verify if $\boldsymbol{f}\!\left(\boldsymbol{x}\right) = \boldsymbol{0}$ is sufficiently satisfied;
- print the value of $V$ in kilovolts;
  and
- print the values of $\delta_{\text{l}}$ and $\delta_{\text{s}}$ in degrees.

### Task 1

Determine $V$, $\delta_{\text{l}}$, and $\delta_{\text{s}}$
when $U$, $F$, $R$, $L$, $C$, $P$, and $\phi$
have respective values of
13.8 kilovolts,
60 hertz,
0.21 ohms, 5.570 millihenrys, 1.560 millifarads,
40 kilowatts, 0.87.

In [3]:
U   = 13.8e3
F   = 60
R   = 0.21
L   = 5.570e-3
C   = 1.560e-3
P   = 40e3
PHI = 0.87

x_ini = np.array([1.2e4, 0.1])

solution = root(f, x_ini, args=(U, F, R, L, C, P, PHI))

x_out = solution.x

residual = f(x_out, U, F, R, L, C, P, PHI)

V = x_out[0] / 1000
delta_l = np.degrees(x_out[1])
delta_s = -delta_l

print("Solution converged:", solution.success)
print("Residual f(x):", residual)
print(f"V = {V:.4f} kV")
print(f"δ_l = {delta_l:.4f} degrees")
print(f"δ_s = {delta_s:.4f} degrees")

Solution converged: True
Residual f(x): [7.85894372e-09 1.71212378e-10]
V = 6.1653 kV
δ_l = -3.1629 degrees
δ_s = 3.1629 degrees


### Task 2

Determine $V$, $\delta_{\text{l}}$, and $\delta_{\text{s}}$
when $U$, $F$, $R$, $L$, $C$, $P$, and $\phi$
have respective values of
13.8 kilovolts,
50 hertz,
0.17 ohms, 6.685 milllihenrys, 1.872 millifarads,
32 kilowatts, 0.97.

In [4]:
U   = 13.8e3
F   = 50
R   = 0.17
L   = 6.685e-3
C   = 1.872e-3
P   = 32e3
PHI = 0.97

x_ini = np.array([1.2e4, 0.1])

solution = root(f, x_ini, args=(U, F, R, L, C, P, PHI))

x_out = solution.x

residual = f(x_out, U, F, R, L, C, P, PHI)

V = x_out[0] / 1000
delta_l = np.degrees(x_out[1])
delta_s = -delta_l

print("Solution converged:", solution.success)
print("Residual f(x):", residual)
print(f"V = {V:.4f} kV")
print(f"δ_l = {delta_l:.4f} degrees")
print(f"δ_s = {delta_s:.4f} degrees")

Solution converged: True
Residual f(x): [3.32056516e-09 9.03810360e-11]
V = 6.1680 kV
δ_l = -2.5611 degrees
δ_s = 2.5611 degrees


### Task 3

Determine $V$, $\delta_{\text{l}}$, and $\delta_{\text{s}}$
when $U$, $F$, $R$, $L$, $C$, $P$, and $\phi$
have respective values of
32.5 kilovolts,
60 hertz,
0.21 ohms, 5.570 millihenrys, 1.560 millifarads,
214 kilowatts, 0.87.

In [5]:
U   = 32.5e3
F   = 60
R   = 0.21
L   = 5.570e-3
C   = 1.560e-3
P   = 214e3
PHI = 0.87

x_ini = np.array([3.0e4, 0.1])

solution = root(f, x_ini, args=(U, F, R, L, C, P, PHI))

x_out = solution.x

residual = f(x_out, U, F, R, L, C, P, PHI)

V = x_out[0] / 1000
delta_l = np.degrees(x_out[1])
delta_s = -delta_l

print("Solution converged:", solution.success)
print("Residual f(x):", residual)
print(f"V = {V:.4f} kV")
print(f"δ_l = {delta_l:.4f} degrees")
print(f"δ_s = {delta_s:.4f} degrees")

Solution converged: True
Residual f(x): [1.66128302e-08 3.63797881e-10]
V = 14.5197 kV
δ_l = -3.1629 degrees
δ_s = 3.1629 degrees


### Task 4

Determine $V$, $\delta_{\text{l}}$, and $\delta_{\text{s}}$
when $U$, $F$, $R$, $L$, $C$, $P$, and $\phi$
have respective values of
32.5 kilovolts,
50 hertz,
0.17 ohms, 6.685 milllihenrys, 1.872 millifarads,
173 kilowatts, 0.97.

In [6]:
U   = 32.5e3
F   = 50
R   = 0.17
L   = 6.685e-3
C   = 1.872e-3
P   = 173e3
PHI = 0.97

x_ini = np.array([3.0e4, 0.1])

solution = root(f, x_ini, args=(U, F, R, L, C, P, PHI))

x_out = solution.x

residual = f(x_out, U, F, R, L, C, P, PHI)

V = x_out[0] / 1000
delta_l = np.degrees(x_out[1])
delta_s = -delta_l

print("Solution converged:", solution.success)
print("Residual f(x):", residual)
print(f"V = {V:.4f} kV")
print(f"δ_l = {delta_l:.4f} degrees")
print(f"δ_s = {delta_s:.4f} degrees")

Solution converged: True
Residual f(x): [7.00856617e-09 1.90993887e-10]
V = 14.5261 kV
δ_l = -2.5611 degrees
δ_s = 2.5611 degrees


### Task 5

Determine $V$, $\delta_{\text{l}}$, and $\delta_{\text{s}}$
when $U$, $F$, $R$, $L$, $C$, $P$, and $\phi$
have respective values of
69.0 kilovolts,
60 hertz,
0.21 ohms, 5.570 millihenrys, 1.560 millifarads,
421 kilowatts, 0.87.

In [7]:
U   = 69.0e3
F   = 60
R   = 0.21
L   = 5.570e-3
C   = 1.560e-3
P   = 421e3
PHI = 0.87

x_ini = np.array([6.5e4, 0.1])

solution = root(f, x_ini, args=(U, F, R, L, C, P, PHI))

x_out = solution.x

residual = f(x_out, U, F, R, L, C, P, PHI)

V = x_out[0] / 1000
delta_l = np.degrees(x_out[1])
delta_s = -delta_l

print("Solution converged:", solution.success)
print("Residual f(x):", residual)
print(f"V = {V:.4f} kV")
print(f"δ_l = {delta_l:.4f} degrees")
print(f"δ_s = {delta_s:.4f} degrees")

Solution converged: True
Residual f(x): [3.41024133e-08 7.49423634e-10]
V = 30.8264 kV
δ_l = -3.1629 degrees
δ_s = 3.1629 degrees


### Task 6

Determine $V$, $\delta_{\text{l}}$, and $\delta_{\text{s}}$
when $U$, $F$, $R$, $L$, $C$, $P$, and $\phi$
have respective values of
32.5 kilovolts,
50 hertz,
0.17 ohms, 6.685 milllihenrys, 1.872 millifarads,
317 kilowatts, 0.97.

In [8]:
U   = 32.5e3
F   = 50
R   = 0.17
L   = 6.685e-3
C   = 1.872e-3
P   = 317e3
PHI = 0.97

x_ini = np.array([3.0e4, 0.1])

solution = root(f, x_ini, args=(U, F, R, L, C, P, PHI))

x_out = solution.x

residual = f(x_out, U, F, R, L, C, P, PHI)

V = x_out[0] / 1000
delta_l = np.degrees(x_out[1])
delta_s = -delta_l

print("Solution converged:", solution.success)
print("Residual f(x):", residual)
print(f"V = {V:.4f} kV")
print(f"δ_l = {delta_l:.4f} degrees")
print(f"δ_s = {delta_s:.4f} degrees")

Solution converged: True
Residual f(x): [6.97946234e-09 1.90993887e-10]
V = 14.5261 kV
δ_l = -2.5611 degrees
δ_s = 2.5611 degrees


## Instructions

Do not use any library or module other than those pre-imported.

### Scoring

In each Task,
- a correct use of
  [`scipy.optimize.root()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.root.html)
  merits two (2) points;
- obtaining a correct $V$ merits two (3) points;
- obtaining a correct $\delta_{\text{l}}$ merits two (3) points;
- obtaining a correct $\delta_{\text{s}}$ merits two (3) points;
  and
- obtaining $\boldsymbol{f}\!\left(\boldsymbol{x}\right)$ close enough to $\boldsymbol{0}$ merits six (9) points.


Every violation of an instruction
means a deduction of one (1) point.

All in all, one may earn up to 120 points for this activity.

### Submission

Download this notebook file,
and save with a filename following the pattern
`ACT-09_<Group name>`,
where the group name is as listed in class.
For example, if you belong to the group SixIsEven,
then your notebook should be named `ACT-09_SixIsEven.ipynb`.
Submit your notebook via the classwork platform for this activity in Google Classroom.
Submissions beyond the deadline will not be considered.

Lastly, the use of AI tools to answer this exam is not prohibited,
but it is of ethical interest to disclose such use.
This is in line with the
[MSU Policy on the Fair and Ethical Use of AI and Its Applications](https://www.msumain.edu.ph/wp-content/uploads/2024/05/MSU-Policy-on-Ethical-use-of-AI-Policies.pdf).
As such, please include a brief statement (in a private comment to this classwork)
declaring which and how AI tools are used in your work.

*Last updated by Christian Cahig on 2025-12-11*